### Code to test single-variable LSTM models trained using NASA weather data.

In [8]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.models import load_model
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

3 Variable

In [12]:
# Fix random seed for reproducibility
np.random.seed(7)

def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # Finding the end of the pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        
        # Checking if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
            
        # Gather input and output parts of pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix: out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def load_col(txtfile, col, process):
    dataset = pd.read_csv(txtfile, sep='\t',
                               header=None,usecols=col)
    if process==True:
        #dataset = dataset.values[2:]
        dataset = dataset.astype('float32')
        scaler = MinMaxScaler(feature_range=(0,1))
        dataset = scaler.fit_transform(dataset)
        return dataset, scaler
    elif process==False:
        return dataset

In [13]:
os.listdir()

['.ipynb_checkpoints',
 'multivariable_lstm_1.0.0.py',
 'multivariable_lstm_2var.slurm',
 'multivariable_lstm_3var.slurm',
 'multivariable_lstm_example_full_predictions.txt',
 'multivariable_lstm_example_full_predictions_no_headers.txt',
 'multivariable_lstm_example_matrix.txt',
 'multivariable_lstm_example_matrix_3850.txt',
 'multivariable_lstm_example_matrix_3850_2.txt',
 'multivariate_lstm_1.0.0.ipynb',
 'multivariate_lstm_1.0.1.ipynb',
 'multivariate_lstm_sample.txt',
 'multivar_3var.h5',
 'multivar_3var_outfile.txt',
 'multivar_kingston_lstm_testmodels.ipynb']

In [15]:
# Load data from text file with selected columns, scaled to (0,1)
F = "multivariate_lstm_sample.txt"
C = [1, 2, 3]
I = 100
O = 100 
data_array, data_scaler = load_col(F, C, True)
#print(data_array[0:10])
n_steps_in = I
n_steps_out = O
dataset = data_array
n_features = 1

print("preparing data...")
X_dataset, y_dataset = split_sequence(data_array, n_steps_in, n_steps_out)
#y_dataset = y_dataset.reshape((y_dataset.shape[0], y_dataset.shape[1]))
print("done!")

print("loading model...")
model = load_model("multivar_3var.h5")
print("done!")

preparing data...
done!
loading model...
done!


In [16]:
print(X_dataset.shape, y_dataset.shape)

(29801, 100, 3) (29801, 100, 3)


In [17]:
yhat = model.predict(X_dataset)

In [18]:
print(yhat.shape)

(29801, 100, 3)


In [37]:
yhat_3751 = yhat[3751]
inv_yhat_pressure_3751 = data_scaler.inverse_transform(yhat_3751)

In [41]:
inv = inv_yhat_pressure_3751
print(inv[0])

[ 28.576313 109.24033  444.60638 ]


In [45]:
outputfilename = "kingston_3var_prediction_3751_to_3850.txt"

output_file = open(outputfilename, "w")

print("Opened txt file")

for i in range(len(inv)):
    #print(i, end='\r')
    entry = str(inv[i][0]) + "\t" + str(inv[i][1]) + "\t" + str(inv[i][2]) + "\n"
    output_file.write(entry)

print("Closed txt file")

Opened txt file
Closed txt file


In [52]:
inv_yhat = []
for i in (range(len(yhat))):
    array = data_scaler.inverse_transform(yhat[i])
    inv_yhat.append(array)

In [57]:
inv_yhat_1 = []
for i in range(len(data_array) - n_steps_in - n_steps_out + 1):
    inv_yhat_1.append(inv_yhat[i][0])

In [60]:
print(len(inv_yhat_1))

29801


In [62]:
outputfilename = "kingston_3var_prediction_full.txt"

output_file = open(outputfilename, "w")

print("Opened txt file")

for i in range(len(inv_yhat_1)):
    #print(i, end='\r')
    entry = str(inv_yhat_1[i][0]) + "\t" + str(inv_yhat_1[i][1]) + "\t" + str(inv_yhat_1[i][2]) + "\n"
    output_file.write(entry)

print("Closed txt file")

Opened txt file
Closed txt file
